<a href="https://colab.research.google.com/github/m0hit-kumar/Simple-Transformers/blob/main/Simple_Trasformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

# Download the SQuAD training dataset
response = requests.get("https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json")

# Save the dataset to a file
with open("train.json", "w") as f:
    f.write(response.content.decode("utf-8"))

# Download the SQuAD development dataset
response = requests.get("https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json")

# Save the dataset to a file
with open("dev.json", "w") as f:
    f.write(response.content.decode("utf-8"))


In [ ]:
data= open("train.json")
test=open("dev.json")

In [ ]:
import json
data=json.load(data)

In [ ]:
dataset=[]
for i in data["data"]:
  for j in i["paragraphs"]:
    dataset.append(j)

In [29]:
!pip install simpletransformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 25.9 MB/s eta 0:00:00


In [ ]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [33]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [31]:
# Configure the model
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5

In [ ]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    # "wandb_project": "Question Answer Application",
    # "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

Explanation of each argument used in your `train_args` dictionary for a Simple Transformers model:

1. `"reprocess_input_data": True`:
   - **Meaning**: If set to `True`, it forces the model to preprocess and tokenize the input data even if a cached version exists. Useful when you want to reprocess the input data.
   - **Use**: It ensures that the data preprocessing step is performed, even if you've previously processed and cached the input data.

2. `"overwrite_output_dir": True`:
   - **Meaning**: If set to `True`, it allows the model to overwrite the output directory if it already exists. Be cautious with this setting as it can result in data loss if not used carefully.
   - **Use**: It ensures that the model can save its output and checkpoints to the specified directory, even if that directory already contains data from a previous run.

3. `"use_cached_eval_features": True`:
   - **Meaning**: If set to `True`, it tells the model to use cached evaluation features instead of recomputing them. Caching can save time when evaluating a model multiple times.
   - **Use**: It speeds up the evaluation process by reusing cached evaluation features when available.

4. `"output_dir": f"outputs/{model_type}"`:
   - **Meaning**: Specifies the directory where the model's output (checkpoints, logs, etc.) will be saved.
   - **Use**: It determines the location where the model will store its outputs during training and evaluation.

5. `"best_model_dir": f"outputs/{model_type}/best_model"`:
   - **Meaning**: Specifies the directory where the best model checkpoints (those with the best performance) will be saved during training.
   - **Use**: It defines the location for storing the best-performing model checkpoints.

6. `"evaluate_during_training": True`:
   - **Meaning**: If set to `True`, it enables evaluation of the model's performance during training at specified intervals.
   - **Use**: It allows you to monitor how the model is performing as it trains and take actions based on evaluation metrics.

7. `"max_seq_length": 128`:
   - **Meaning**: Sets the maximum sequence length for tokenized input data. Sequences longer than this length will be truncated or split.
   - **Use**: It controls how long the input sequences can be and is important for managing memory usage during training.

8. `"num_train_epochs": 5`:
   - **Meaning**: Specifies the number of training epochs (complete passes through the training data) for the model.
   - **Use**: It determines how many times the model will go through the entire training dataset during training.

9. `"evaluate_during_training_steps": 1000`:
   - **Meaning**: Sets the frequency (in steps) at which evaluation is performed during training.
   - **Use**: It controls how often the model's performance is evaluated while it's training.

10. `"wandb_project": "Question Answer Application"`:
    - **Meaning**: Specifies the name of the WandB project for logging and monitoring training progress.
    - **Use**: It connects the training process to the specified WandB project for tracking experiments and visualizations.

11. `"wandb_kwargs": {"name": model_name}`:
    - **Meaning**: Additional arguments to pass to WandB for configuring the run. In this case, it sets the name of the run.
    - **Use**: It customizes the WandB run name to help identify and organize experiments.

12. `"save_model_every_epoch": False`:
    - **Meaning**: If set to `True`, it saves the model's checkpoint after each training epoch.
    - **Use**: It determines whether or not to save a model checkpoint at the end of each training epoch.

13. `"save_eval_checkpoints": False`:
    - **Meaning**: If set to `True`, it saves model checkpoints after each evaluation during training.
    - **Use**: It controls whether the model checkpoints are saved at each evaluation step.

14. `"n_best_size": 3`:
    - **Meaning**: Specifies the number of best predictions to consider when evaluating performance.
    - **Use**: It determines how many of the best model predictions to consider when evaluating the model.

15. `"use_early_stopping": True`:
    - **Meaning**: If set to `True`, it enables early stopping based on the specified metric.
    - **Use**: It allows the training to stop early if the model's performance on the chosen metric does not improve.

16. `"early_stopping_metric": "mcc"`:
    - **Meaning**: Defines the metric used for early stopping. In this case, it's using Matthews correlation coefficient (mcc).
    - **Use**: It specifies which metric to monitor for early stopping.

17. `"n_gpu": 2`:
    - **Meaning**: Sets the number of GPUs to use for training (parallelization).
    - **Use**: It controls the number of GPUs that the model can use for faster training if available.

18. `"manual_seed": 4`:
    - **Meaning**: This argument sets a specific manual seed value for the random number generator used in various parts of the training process, such as weight initialization and data shuffling. Setting a manual seed ensures that the random processes are reproducible, and you get the same results every time you run the training.

    - **Use**:  It is commonly used when you want to ensure that your model's training process is deterministic, which is important for reproducibility. By setting a fixed seed (in this case, 4), you ensure that the randomness introduced during training (e.g., weight initialization) follows the same pattern across different runs of the training process.



19. `"use_multiprocessing": False`:
    - **Meaning**: If set to `True`, it enables multiprocessing for data loading during training.
    - **Use**: It can speed up data loading by utilizing multiple CPU cores. Set to `False` if you encounter issues related to multiprocessing.

20. `"train_batch_size": 128`:
    - **Meaning**: Specifies the batch size used during training. It determines how many samples are processed in each forward and backward pass.
    - **Use**: It controls the size of mini-batches used for training. A larger batch size can speed up training but may require more memory.

21. `"eval_batch_size": 64`:
    - **Meaning**: Specifies the batch size used during evaluation. It determines how many samples are processed in each evaluation step.
    - **Use**: Similar to `train_batch_size`, it controls the size of mini-batches used for evaluation.

22. `"config": {"output_hidden_states": True}`:
    - **Meaning**: A dictionary used to configure specific model settings. In this case, it sets `output_hidden_states` to `True`.
    - **Use**: It customizes the model's behavior by setting specific configuration options. In this case, it enables the model to output hidden states during inference.


In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the name of the GPU device
    device = torch.cuda.get_device_name(0)
    print("CUDA device:", device)
else:
    print("CUDA is not available.")


CUDA device: Tesla T4


In [ ]:
model = QuestionAnsweringModel(model_type,model_name, args=model_args,use_cuda=True)

NameError: ignored

In [ ]:
# Train the model
model.train_model(dataset, eval_data=random.sample(dataset,50))

add example index and unique id: 100%|██████████| 130319/130319 [00:00<00:00, 736878.76it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/8229 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


convert squad examples to features:   0%|          | 0/322 [00:00<?, ?it/s]

convert squad examples to features: 100%|██████████| 322/322 [00:01<00:00, 185.26it/s]


add example index and unique id: 100%|██████████| 322/322 [00:00<00:00, 493087.22it/s]


Running Evaluation:   0%|          | 0/41 [00:00<?, ?it/s]

...



convert squad examples to features:   0%|          | 0/322 [00:00<?, ?it/s]

convert squad examples to features: 100%|██████████| 322/322 [00:01<00:00, 191.93it/s]


add example index and unique id: 100%|██████████| 322/322 [00:00<00:00, 468458.51it/s]


Running Evaluation:   0%|          | 0/41 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 322/322 [00:01<00:00, 191.02it/s]

add example index and unique id: 100%|██████████| 322/322 [00:00<00:00, 425777.39it/s]


Running Evaluation:   0%|          | 0/41 [00:00<?, ?it/s]

(41145,
 {'global_step': [2000,
   4000,
   6000,
   8000,
   8229,
   10000,
   12000,
   14000,
   16000,
   16458,
   18000,
   20000,
   22000,
   24000,
   24687,
   26000,
   28000,
   30000,
   32000,
   32916,
   34000,
   36000,
   38000,
   40000,
   41145],
  'correct': [144,
   169,
   179,
   185,
   185,
   193,
   205,
   213,
   208,
   220,
   221,
   226,
   228,
   236,
   239,
   236,
   245,
   250,
   249,
   250,
   250,
   254,
   259,
   259,
   258],
  'similar': [154,
   131,
   128,
   126,
   123,
   114,
   109,
   105,
   109,
   96,
   93,
   94,
   88,
   83,
   82,
   83,
   74,
   71,
   72,
   71,
   71,
   68,
   62,
   63,
   64],
  'incorrect': [24,
   22,
   15,
   11,
   14,
   15,
   8,
   4,
   5,
   6,
   8,
   2,
   6,
   3,
   1,
   3,
   3,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   0],
  'train_loss': [1.2437645196914673,
   1.5939507484436035,
   1.3561859130859375,
   0.5229277610778809,
   1.6326671838760376,
   0.6638693809509277,
 

In [ ]:
result, texts = model.eval_model(dataset[:100])

add example index and unique id: 100%|██████████| 1165/1165 [00:00<00:00, 617667.07it/s]


Running Evaluation:   0%|          | 0/151 [00:00<?, ?it/s]

In [ ]:
for i in texts["similar_text"]:
  if texts["similar_text"][i]["predicted"] != '':
    print(texts["similar_text"][i])


{'truth': 'in the late 1990s', 'predicted': 'late 1990s', 'question': 'When did Beyonce start becoming popular?'}
{'truth': 'Men in Black.', 'predicted': 'Men in Black', 'question': "Destiny's Child song, Killing Time, was included in which film's soundtrack?"}
{'truth': 'split with Luckett and Rober', 'predicted': 'split with Luckett and Roberson', 'question': "What event caused Beyonce's depression?"}
{'truth': 'Farrah Franklin and Michelle Williams.', 'predicted': 'Farrah Franklin', 'question': "Who replaced Luckett and Roberson in Destiny's Child?"}
{'truth': 'Fighting Temptations', 'predicted': 'The Fighting Temptations', 'question': 'What song did Beyoncé release as the lead single from The Fighting Tempations?'}
{'truth': 'ten nominations', 'predicted': 'ten', 'question': 'Beyonce received how many nominations at the 52nd Annual Grammy Awards?'}
{'truth': 'documents obtained by WikiLeaks', 'predicted': 'documents', 'question': 'Hoe did everyone learn that Beyonce performed for K

In [ ]:
to_predict = [
    {
        "context": dataset[0]["context"],
        "qas": [
            {
                "question": "What was the name of Beyoncé's first solo album?",
                "id": "56bf6b0f3aeaaa14008c9604",
            }
        ],
    }
]

NameError: ignored

In [ ]:
answers, probabilities = model.predict(to_predict)
print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 12192.74it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '56bf6b0f3aeaaa14008c9604', 'answer': ['', 'Beyoncé Giselle Knowles-Carter']}]


In [38]:
import gdown
shared_drive_link = 'https://drive.google.com/uc?id=1BduCqWQv0suOSKH_mc_0BVw5rvpAArbJ'

output_path = 'PytorchModel.zip'

# Download the zip file
gdown.download(shared_drive_link, output_path, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1BduCqWQv0suOSKH_mc_0BVw5rvpAArbJ
To: /content/PytorchModel.zip
100%|██████████| 1.17G/1.17G [00:15<00:00, 75.3MB/s]


'PytorchModel.zip'

In [41]:
!ls

 1BduCqWQv0suOSKH_mc_0BVw5rvpAArbJ   PytorchModel.zip  'view?usp=sharing'
 drive				     sample_data


In [40]:
!rm -rf PytorchModel

In [42]:
!unzip -q PytorchModel.zip -d PytorchModel


In [43]:
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

# Create a QuestionAnsweringModel object
model = QuestionAnsweringModel(model_type,"PytorchModel",args=model_args,use_cuda=True)


In [44]:
to_predict = [
    {
        "context": "She released her first solo album, Dangerously in Love (2003), during the hiatus of Destiny's Child. It debuted at number one on the US Billboard 200 and produced the singles Crazy in Love, Baby Boy, Me, Myself and I, Naughty Girl and The Closer I Get to You",
        "qas": [
            {
                "question": "What was the name of Beyoncé's first solo album?",
                "id": "56bf6b0f3aeaaa14008c9604",
            }
        ],
    }
]

In [45]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 9776.93it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '56bf6b0f3aeaaa14008c9604', 'answer': ['Dangerously in Love', 'Dangerously in Love (2003)', 'Love']}]
